<a href="https://colab.research.google.com/github/SHIVRAJSHINDE/Colab01/blob/main/Solution_HuggingFace_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip -q install accelerate -U
!pip -q install transformers[torch]
!pip -q install datasets
!pip install --upgrade pyarrow
#Restart after installing


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from transformers import pipeline
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, ClassLabel, Dataset


In [23]:
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Final_Emotion_Data/five_emotions_data.csv
emotions_data=pd.read_csv("five_emotions_data.csv")
print(emotions_data.shape)
print(emotions_data.head())
print(emotions_data["label"].value_counts())


--2024-12-29 14:10:23--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Final_Emotion_Data/five_emotions_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4092906 (3.9M) [text/plain]
Saving to: ‘five_emotions_data.csv.5’

five_emotions_data. 100%[===================>]   3.90M  --.-KB/s    in 0.07s   

2024-12-29 14:10:23 (59.4 MB/s) - ‘five_emotions_data.csv.5’ saved [4092906/4092906]

(42645, 4)
   Id    Emotion                                               Text  label
0   1    sadness                            i didnt feel humiliated      3
1   2    sadness  i can go from feeling so hopeless to so damned...      3
2   4       love  i am ever feeling nostalgic about the fireplac...      1
3   6    sadness  ive been feeling a 

In [24]:
# Distil bert model
from transformers import pipeline
distilbert_model = pipeline(task="text-classification",
                            model="distilbert-base-uncased",
                            device="cuda",
                            )


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda


In [25]:

sample_data=emotions_data.sample(10000, random_state=42)
sample_data["Text"]=sample_data["Text"].apply(lambda x: " ".join(x.split()[:100]))
sample_data["bert_predicted"] = sample_data["Text"].apply(lambda x: distilbert_model(x)[0]["label"])
sample_data["bert_predicted_num"]=sample_data["bert_predicted"].apply(lambda x: x[-1])
sample_data["bert_predicted_num"] = sample_data["bert_predicted_num"].astype(int)
sample_data.head()



,Id,Emotion,Text,label,bert_predicted,bert_predicted_num
1900,2690,sadness,i social and dreaming about things that make y...,3,LABEL_1,1
20627,27544,worry,"is missing training tonight, the lurgy is on m...",4,LABEL_0,0
12481,17415,worry,my HD is full. need to cleanup a lot,4,LABEL_0,0
30267,39810,sadness,"i'm watching missing pieces, just coz the them...",3,LABEL_0,0
14420,19838,sadness,rain got so big weather so cold right now,3,LABEL_1,1


In [26]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(sample_data["label"], sample_data["bert_predicted_num"])
print(cm)
accuracy=cm.diagonal().sum()/cm.sum()
print(accuracy)


[[1380 1142    0    0    0]
 [ 774  481    0    0    0]
 [1273  671    0    0    0]
 [1171 1081    0    0    0]
 [1365  662    0    0    0]]
0.1861


In [27]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, ClassLabel, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch


In [28]:
Sample_data = Dataset.from_pandas(sample_data)
# Split the dataset into training and testing sets

train_test_split = Sample_data.train_test_split(test_size=0.2)  # 80% training, 20% testing

dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})
dataset


DatasetDict({
    train: Dataset({
        features: ['Id', 'Emotion', 'Text', 'label', 'bert_predicted', 'bert_predicted_num', '__index_level_0__'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['Id', 'Emotion', 'Text', 'label', 'bert_predicted', 'bert_predicted_num', '__index_level_0__'],
        num_rows: 2000
    })
})

In [29]:
# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Padding
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.add_special_tokens({'pad_token': '[PAD]'} )

def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=100)
tokenized_datasets = dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
import os
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Disable WandB logging
os.environ["WANDB_DISABLED"] = "true"

# Load model
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=5,
    pad_token_id=tokenizer.eos_token_id
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results_bert_custom",
    num_train_epochs=5,
    logging_dir="./logs_bert_custom",
    eval_strategy="epoch",
    report_to=[],  # Disable WandB logging
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# Start training
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.065000,0.993877
2,0.836000,1.039340
3,0.603100,1.233816
4,0.353900,1.636532
5,0.194400,2.025911


TrainOutput(global_step=5000, training_loss=0.6263781005859375, metrics={'train_runtime': 535.0078, 'train_samples_per_second': 74.765, 'train_steps_per_second': 9.346, 'total_flos': 1034956920000000.0, 'train_loss': 0.6263781005859375, 'epoch': 5.0})

In [42]:
# Define the directory where you want to save your model and tokenizer
model_dir = "./distilbert_finetuned"

# Save the model
model.save_pretrained(model_dir)

# Save the tokenizer
tokenizer.save_pretrained(model_dir)

#Save the model with
trainer.save_model('Distilbert_CustomModel_10K')

#!zip -r distilbert_finetuned_10k.zip ./distilbert_finetuned


In [43]:
def make_prediction(text):
  new_text=text
  inputs=tokenizer(new_text, return_tensors="pt")
  inputs = inputs.to(torch.device("cuda:0"))
  outputs=model(**inputs)
  predictions=outputs.logits.argmax(-1)
  predictions=predictions.detach().cpu().numpy()
  return(predictions)

sample_data["finetuned_predicted"]=sample_data["Text"].apply(lambda x: make_prediction(str(x))[0])



In [44]:
sample_data["finetuned_predicted"]


,finetuned_predicted
1900,3
20627,4
12481,4
30267,3
14420,3
...,...
35744,1
36414,4
8101,3
27367,2


In [45]:
from sklearn.metrics import confusion_matrix
# Create the confusion matrix
cm1 = confusion_matrix(sample_data["label"], sample_data["finetuned_predicted"])
print(cm1)
accuracy1=cm1.diagonal().sum()/cm1.sum()
print(accuracy1)


[[2315   76   75   22   34]
 [  85 1096   29   19   26]
 [  53   38 1687   45  121]
 [  17   10   49 2078   98]
 [  38   33  104   90 1762]]
0.8938
